In [124]:
import pandas as pd
import os
import numpy as np

In [ ]:
# Caminhos padrões para os arquivos e pastas
caminho_raw = "../dados/raw"
caminho_processed = "../dados/processed"
caminho_gold = "../dados/gold"

In [ ]:
# Visualização das colunas de dados 'float' para somente 2 casas decimais
pd.set_option('display.float_format', '{:,.2f}'.format)

In [ ]:
# leitura dos dados do dre
df_dre = pd.read_parquet(os.path.join(caminho_raw,'dre_raw.parquet'))

# Renomeação das colunas para facilitar visualização e direcionamentos das melhores práticos python
colunas_renomeadas = {
    'Instituição': 'instituicao',
    'Código': 'codigo_bacen',
    'Conglomerado Prudencial': 'conglomerado',
    'TCB': 'tcb',
    'TC': 'tc',
    'TI': 'ti',
    'Cidade': 'cidade',
    'UF': 'uf',
    'Data': 'data_base',
    'Resultado de Intermediação Financeira': 'resultado_intermediacao',
    'Resultado com Transações de Pagamento': 'resultado_pagamentos',
    'Outras Receitas / Despesas': 'outras_receitas_despesas',
    'Resultado antes da Tributação e Participações (x) = (k) + (l) + (w)': 'resultado_antes_impostos', # LAIR
    'Imposto de Renda e Contribuição Social (y)': 'impostos_ir_csll',
    'Participações no Lucro (z)': 'participacoes_lucro',
    'Lucro Líquido (aa) = (x) + (y) + (z)': 'lucro_liquido'
}

df_dre = df_dre.rename(columns=colunas_renomeadas)

# Transformação dos tipos de dados (float, categoria, string, integer, date_time). Além de tranformar valores não informados (NI) para zero (0), para deixar no formato adequado.
colunas_para_float = [
    'resultado_intermediacao',
    'resultado_pagamentos',
    'outras_receitas_despesas',
    'impostos_ir_csll',
    'participacoes_lucro',
    'resultado_antes_impostos',
    'lucro_liquido'
]

cols_id = ['codigo_bacen', 'tc', 'ti']

cols_categoria = ['uf', 'tcb','conglomerado']

cols_str = ['instituicao','cidade']

for col in colunas_para_float:
    df_dre[col] = df_dre[col].astype(str).str.replace('.', '', regex=False)
    df_dre[col] = pd.to_numeric(df_dre[col], errors='coerce')

for col in cols_id:
    df_dre[col] = pd.to_numeric(df_dre[col], errors='coerce').astype('Int64')

for col in cols_categoria:
    df_dre[col] = df_dre[col].astype('category')

for col in cols_str:
    df_dre[col] = df_dre[col].astype('string')

df_dre['data_base'] = pd.to_datetime(df_dre['data_base'], format='%m/%Y', errors='coerce')

df_dre = df_dre.replace(['NI', 'Ni', 'ni'], np.nan)

# Salvar arquivo DRE transformado
df_dre.to_parquet(os.path.join(caminho_processed,"dre_processed.parquet"))

df_dre.info()
df_dre.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17165 entries, 0 to 17164
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   instituicao               17165 non-null  string        
 1   codigo_bacen              17165 non-null  Int64         
 2   conglomerado              8361 non-null   category      
 3   tcb                       17165 non-null  category      
 4   tc                        17165 non-null  Int64         
 5   ti                        17165 non-null  Int64         
 6   cidade                    17165 non-null  string        
 7   uf                        17165 non-null  category      
 8   data_base                 17165 non-null  datetime64[ns]
 9   resultado_intermediacao   15476 non-null  float64       
 10  resultado_pagamentos      17081 non-null  float64       
 11  outras_receitas_despesas  15476 non-null  float64       
 12  resultado_antes_im

,instituicao,codigo_bacen,conglomerado,tcb,tc,ti,cidade,uf,data_base,resultado_intermediacao,resultado_pagamentos,outras_receitas_despesas,resultado_antes_impostos,impostos_ir_csll,participacoes_lucro,lucro_liquido
0,BANCO DO BRASIL S.A.,0,BB - PRUDENCIAL,b1,1,1,BRASILIA,DF,2022-12-01,"58,297,195.00",0.00,"6,287,062.00","21,753,526.00","-2,982,566.00","-2,132,589.00","16,638,371.00"
1,CAIXA ECONOMICA FEDERAL,360305,CAIXA ECONÔMICA FEDERAL - PRUDENCIAL,b1,1,6,BRASILIA,DF,2022-12-01,"56,235,680.00",0.00,"8,480,850.00","5,582,178.00","971,396.00","-1,060,139.00","5,493,435.00"
2,ITAÚ UNIBANCO S.A.,60701190,ITAU - PRUDENCIAL,b1,2,8,SAO PAULO,SP,2022-12-01,"36,162,795.00",0.00,"4,631,069.00","18,299,359.00","1,334,900.00","-3,690,333.00","15,943,927.00"
3,BANCO BRADESCO S.A.,60746948,BRADESCO - PRUDENCIAL,b1,2,8,OSASCO,SP,2022-12-01,"44,706,379.00",0.00,"6,034,638.00","2,897,870.00","3,749,785.00",0.00,"6,647,656.00"
4,BANCO SANTANDER (BRASIL) S.A.,90400888,SANTANDER - PRUDENCIAL,b1,3,8,SAO PAULO,SP,2022-12-01,"26,385,861.00",0.00,"5,637,941.00","2,602,071.00","2,970,185.00","-990,117.00","4,582,139.00"


In [ ]:
# leitura dos dados do RESUMO
df_resumo = pd.read_parquet(os.path.join(caminho_raw, "resumo_raw.parquet"))

# Renomeação das colunas para facilitar visualização e direcionamentos das melhores práticos python
colunas_para_renomear = {
    'Instituição': 'instituicao',
    'Código': 'codigo_bacen',
    'Conglomerado Prudencial': 'conglomerado',
    'TCB': 'tcb',
    'TC': 'tc',
    'TI': 'ti',
    'Cidade': 'cidade',
    'UF': 'uf',
    'Data': 'data_base',
    'Ativo Total' : 'ativo_total',
    'Captações' : 'captacoes',
    'Patrimônio Líquido' : 'patrimonio_liquido',
    'Lucro Líquido' : 'lucro_liquido',
    'Número de Agências' : 'numero_agencias',
    'Número de Postos de Atendimento' : 'numero_postos_atendimento',
    'Carteira de Crédito' : 'carteira_credito',
    'Passivo Exigível' : 'passivel_exigivel'
}

df_resumo = df_resumo.rename(columns=colunas_para_renomear)

# Transformação dos tipos de dados (float, categoria, string, integer, date_time). Além de tranformar valores não informados (NI) para zero (0), para deixar no formato adequado.
colunas_float = [
    'ativo_total',
    'captacoes',
    'patrimonio_liquido',
    'lucro_liquido',
    'numero_agencias',
    'numero_postos_atendimento',
    'carteira_credito',
    'passivel_exigivel'
]

cols_id = ['codigo_bacen', 'tc', 'ti']

cols_categoria = ['uf', 'tcb','conglomerado']

cols_str = ['instituicao','cidade']

for col in colunas_float:
    df_resumo[col] = df_resumo[col].astype(str).str.replace('.', '', regex=False)
    df_resumo[col] = pd.to_numeric(df_resumo[col], errors='coerce')

for col in cols_id:
    df_resumo[col] = pd.to_numeric(df_resumo[col], errors='coerce').astype('Int64')

for col in cols_categoria:
    df_resumo[col] = df_resumo[col].astype('category')

for col in cols_str:
    df_resumo[col] = df_resumo[col].astype('string')

df_resumo['data_base'] = pd.to_datetime(df_resumo['data_base'], format='%m/%Y', errors='coerce')

df_resumo = df_resumo.replace(['NI', 'Ni', 'ni'], np.nan)

# Salvar arquivo DRE transformado
df_resumo.to_parquet(os.path.join(caminho_processed, "resumo_processed.parquet"))

df_resumo.info()
df_resumo.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17170 entries, 0 to 17169
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   instituicao                17170 non-null  string        
 1   codigo_bacen               17170 non-null  Int64         
 2   conglomerado               8373 non-null   category      
 3   tcb                        17170 non-null  category      
 4   tc                         17170 non-null  Int64         
 5   ti                         17170 non-null  Int64         
 6   cidade                     17170 non-null  string        
 7   uf                         17170 non-null  category      
 8   data_base                  17170 non-null  datetime64[ns]
 9   ativo_total                17042 non-null  float64       
 10  captacoes                  17042 non-null  float64       
 11  patrimonio_liquido         17042 non-null  float64       
 12  lucr

,instituicao,codigo_bacen,conglomerado,tcb,tc,ti,cidade,uf,data_base,ativo_total,captacoes,patrimonio_liquido,lucro_liquido,numero_agencias,numero_postos_atendimento,carteira_credito,passivel_exigivel
0,BANCO DO BRASIL S.A.,0,BB - PRUDENCIAL,b1,1,1,BRASILIA,DF,2022-12-01,"1,927,604,248.00","1,551,106,608.00","153,546,924.00","16,638,371.00",3985,604,"853,636,272.00","1,774,057,324.00"
1,ITAÚ UNIBANCO S.A.,60701190,ITAU - PRUDENCIAL,b1,2,8,SAO PAULO,SP,2022-12-01,"1,621,520,167.00","1,323,163,687.00","129,564,325.00","15,943,927.00",2584,1003,"406,186,786.00","1,491,955,842.00"
2,CAIXA ECONOMICA FEDERAL,360305,CAIXA ECONÔMICA FEDERAL - PRUDENCIAL,b1,1,6,BRASILIA,DF,2022-12-01,"1,585,795,602.00","1,315,128,052.00","84,813,538.00","5,493,435.00",3373,937,"1,012,344,567.00","1,500,982,064.00"
3,BANCO BRADESCO S.A.,60746948,BRADESCO - PRUDENCIAL,b1,2,8,OSASCO,SP,2022-12-01,"1,489,174,462.00","1,147,488,387.00","154,263,223.00","6,647,656.00",2865,1277,"543,700,762.00","1,334,911,238.00"
4,BANCO SANTANDER (BRASIL) S.A.,90400888,SANTANDER - PRUDENCIAL,b1,3,8,SAO PAULO,SP,2022-12-01,"986,299,576.00","722,481,108.00","81,076,905.00","4,582,139.00",2573,1557,"341,933,356.00","905,222,672.00"


In [ ]:
# Criação de um novo dataframe 'analitico', com a junção de resumo e dre, facilitando visualização, análise e correlação.
df_analitico = pd.merge(
    df_resumo, 
    df_dre, 
    on=['codigo_bacen', 'data_base'], 
    how='inner',
    suffixes=('', '_dre') # indicar as colunas que estão em ambos dataframes, colocando o sufixo do df_dre
)

# Removendo colunas que ficaram duplicadas.
colunas_remover = [c for c in df_analitico.columns if '_dre' in c]
df_analitico = df_analitico.drop(columns=colunas_remover)

# Salvar dataframe gold, no qual utilizaremos para análises e novas features.
df_analitico.to_parquet(os.path.join(caminho_gold, "df_gold.parquet"))